# ヒストグラム記述子の抽出

<br>
<div style="text-align: right;">
作成日2022年5月6日<br>                                                                     　作成者：小山翼


In [1]:
import re
import os
import numpy as np
import argparse
import math
import subprocess as sp
from subprocess import check_output
import pandas as pd
import shutil
import make_hist_in_ver2
import glob

【要ユーザー操作】　入力ファイル名の記述　　testdata.csv  を自身のファイル名に変更する

In [2]:
df = pd.read_csv('testdata_test.csv')#ここでファイル名を変更する
print ("number of samples",len(df))
df.head()

number of samples 3


,Index#,formula,EA (eV),S/U
0,1,LiCaSbO4,NaN,NaN
1,2,LiMgPO4,0.30302,S
2,3,LiSrPO4,NaN,NaN


【要ユーザー操作】　組成式が入っているラベルを指定する

In [3]:
%%time
path=os.getcwd()
d_list=[]
str_list=[]

for i in range(len(df)):
    formula="formula"
    histdef='histdef.dat'
    defelem='DefElem'
    defvasp='DefVASP'
    composition=df[formula][i]#ここで組成式が入っているラベルを指定

    parser = argparse.ArgumentParser(
    usage='python make_hist.py structure -his= -sigma= -kh= -pos='
        'composition         : enter a composition name (e.g. LiCoO2) or POSCAR.names formatted file (if -pos=T)'
        '-his=histdef.dat  : histdef.dat formatted strucure file (def. histdef.dat)'
        '-sigma=T          : sigma value for Gaussian broadning'
        '-kh=T             : output histgram descriptors considering two elements (Def. F)'
        '-pos=T            : use POSCAR.names file instead of direct input.(Def. F)'
        'Histogram format  : 1:Prop (eg. AN, EN)  2: 3  3: prop_min  4: prop_max  5: bin_num  6: gaussian_sigma')

    parser.add_argument('-his',type=str, default=histdef)
    parser.add_argument('composition',type=str)
    parser.add_argument('-sigma')
    parser.add_argument('-kh')
    parser.add_argument('-pos',default='F')

    args = parser.parse_args(args=['-his=histdef.dat',composition,'-kh=T'])#ここでオプションを設定する。
    reference = args.composition
    histdef = args.his
    aparam2=args.sigma
    aparam3=args.kh
    sparam=args.pos
    reference = reference.split()
    if aparam2 is None:
        aparam2 = ''
    
    if aparam3 is None:
        aparam3 = ''
    df_columns=[]
    df_data=[]
    mk_hist=make_hist_in_ver2.compdescript(defelem,aparam2,sparam,aparam3,reference)
    comp_hist=["AN","EN","MP","PN","PG","MN","AW","AR","IR","CoR","CrR","spdf"]
    
    comp_list=[]
    comp_list1=[]
    comp_list2=[]
    for comp in comp_hist:
        comp_list.append(mk_hist[3][comp])
        comp_list1.append(mk_hist[4][comp])
        comp_list2.append(mk_hist[5][comp])
    
    df_last1=pd.concat(comp_list, axis=1)
    df_last2=pd.concat(comp_list1, axis=1)
    df_last3=pd.concat(comp_list2, axis=1)
            
    df_h = pd.concat([df_last1,df_last2,df_last3,mk_hist[0],mk_hist[1],mk_hist[2]], axis=1)
    d_list.append(df_h)
    str_list.append(reference[0])
#df2=pd.DataFrame(d_list,index=[i for i in range(len(df))])

df2=pd.concat(d_list,axis=0,ignore_index=True)
df2
#df_new=pd.merge(df,df2,left_index=True, right_index=True)
df_new=pd.concat([df,df2],axis=1)

df_new

Wall time: 3.43 s


,Index#,formula,EA (eV),S/U,AN_0,AN_1,AN_2,AN_3,AN_4,AN_5,...,SPDFmatrix_0,SPDFmatrix_1,SPDFmatrix_2,SPDFmatrix_3,SPDFmatrix_4,SPDFmatrix_5,SPDFmatrix_6,SPDFmatrix_7,SPDFmatrix_8,SPDFmatrix_9
0,1,LiCaSbO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,LiMgPO4,0.30302,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,LiSrPO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000


【要ユーザー操作】　出力するファイル名入力のこと

In [4]:
df_new.to_csv('comp.csv',index=False)